# La música en la Catedral de Santo Domingo de la Calzada
## Aplicación para buscar las obras de los autores
Vamos a presentar una sencilla página web que, dado un autor, nos devuelva todas sus obras clasificadas.
- **Nota:** algunos autores tienen distinto el nombre en las páginas y en el índice onomástico. Por ejemplo el autor *Prado, Ramón José* en las páginas del libro aparece como *Prado*, por lo que habrá que buscarlo por este segundo nombre (el de las páginas).

In [1]:
import numpy as np
import cv2
import pickle
import re,string
from unicodedata import normalize
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [7]:
%%capture
!wget https://www.dropbox.com/s/u2ekoltvu35kfjw/Fotos.zip?dl=1 -O Fotos.zip

In [8]:
%%capture
!unzip Fotos.zip

In [9]:
with open("Obras_final.pickle", "rb") as f:
    Obras_final = pickle.load(f)
with open("ini_Obras.pickle", "rb") as f:
    ini_Obras = pickle.load(f)

In [10]:
def remove_punctuation(frase):
  char = re.sub('[%s]' % re.escape(string.punctuation.replace('-','')),'',frase)
  char = char.replace('¿','').replace('?','').replace('`','').replace('´','').replace('#','').lower()
  char = re.sub(
        r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
        normalize( "NFD", char), 0, re.I
    )
  char = normalize( 'NFC', char)
  return char

In [11]:
claves = [pag for pag in ini_Obras]
Pagina = {}
for cla in claves:
  foto = cv2.imread('Fotos/pag'+str(cla)+'.jpg')
  foto = cv2.cvtColor(foto,cv2.COLOR_BGR2RGB)
  Pagina[cla] = foto

In [12]:
Autores = list(Obras_final.keys())
Autores2 = [remove_punctuation(a) for a in Autores]
Autores22 = []
for a in Autores:
  if len(a.split(sep=',')) > 1:
    Autores22.append(remove_punctuation(' '.join(a.split(sep=',')[::-1]))[1:])
  else:
    Autores22.append(remove_punctuation(' '.join(a.split(sep=',')[::-1])))

In [13]:
def verPagina(num_obra):
  for cla in ini_Obras:
    if num_obra in ini_Obras[cla]: return cla

In [14]:
def buscar(autor):
  if autor not in Autores + Autores2 + Autores22: print('Ese autor no existe o está mal escrito.')
  else:
    if autor in Autores2:
      autor = Autores[Autores2.index(autor)]
    elif autor in Autores22:
      autor = Autores[Autores22.index(autor)]
    Foto = np.zeros((100,1000,3),dtype='uint8') + 255
    cv2.putText(Foto, 'Aqui estan las obras del autor: ' + remove_punctuation(autor) + '.', (100, 60), cv2.FONT_HERSHEY_TRIPLEX,0.8, (0, 0, 0), 2)
    fig, ax = plt.subplots(1,1,figsize=(16,16))
    ax.imshow(Foto,cmap=plt.cm.binary)
    plt.show()
    for i in Obras_final[autor]:
      Foto = []
      pag = verPagina(i)
      n = len(Obras_final[autor][i]['Coordenadas'])
      for j in range(0,n):
        x,y,w,h = Obras_final[autor][i]['Coordenadas'][j]
        Foto.append(Pagina[pag+j][y:y+h+5,x:x+w])
      Foto = np.concatenate([np.zeros((100,1080,3),dtype='uint8')+255]+Foto,axis=0)
      types = 'Tipos: ' + ', '.join([remove_punctuation(t) for t in Obras_final[autor][i]['Tipo']]) + '.'
      cv2.rectangle(Foto, (0,0), (1080,50), (119, 90, 17), 2)
      cv2.putText(Foto, types, (150, 30), cv2.FONT_HERSHEY_TRIPLEX,0.6, (0, 0, 0), 2)
      cv2.putText(Foto, 'Pagina: ' + str(pag), (500, 30), cv2.FONT_HERSHEY_TRIPLEX,0.6, (0, 0, 0), 2)
      fig, ax = plt.subplots(1,1,figsize=(16,16))
      ax.imshow(Foto,cmap=plt.cm.binary)
      plt.show()

In [15]:
keyword=widgets.Text(
    placeholder='Autor: ',
    disabled=False
)
display(keyword)


button = widgets.Button(description="Buscar")
display(button)

output = widgets.Output()
@output.capture()
def on_button_clicked(b):
  output.clear_output()
  if keyword.value == '': 
    print('No has escrito nada.')
  else:
    buscar(keyword.value)

button.on_click(on_button_clicked)
display(output)

Text(value='', placeholder='Autor: ')

Button(description='Buscar', style=ButtonStyle())

Output()